In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itenet" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:14<00:00,  5.28s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  54%|█████▍    | 14/26 [00:00<00:00, 131.88batch/s, loss=0.234]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9171 | AUPR: 0.6566 | Acc: 95.32%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9204 | AUPR: 0.6639 | Acc: 95.36%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9224 | AUPR: 0.6690 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9165 | AUPR: 0.6575 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9202 | AUPR: 0.6626 | Acc: 95.37%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9216 | AUPR: 0.6667 | Acc: 95.42%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9224 | AUPR: 0.6690 | Acc: 95.33%
Internal test: AUC: 0.9301 | AUPR: 0.7001 | Acc: 95.28%
External test: AUC: 0.9231 | AUPR: 0.6688 | Acc: 95.17%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9168 | AUPR: 0.6217 | Acc: 94.98%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9172 | AUPR: 0.6187 | Acc: 95.04%
model(kernel=linear, C=1)
Valid:         AUC: 0.9163 | AUPR: 0.6129 | Acc: 95.03%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9172 | AUPR: 0.6187 | Acc: 95.04%
Internal test: AUC: 0.9260 | AUPR: 0.6685 | Acc: 95.04%
External test: AUC: 0.9166 | AUPR: 0.6116 | Acc: 94.66%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7551 | AUPR: 0.4207 | Acc: 94.63%
model(n_neighbors=5)
Valid:         AUC: 0.7840 | AUPR: 0.4738 | Acc: 95.02%
model(n_neighbors=7)
Valid:         AUC: 0.8025 | AUPR: 0.4976 | Acc: 95.04%
best model(n_neighbors=7)
Valid:         AUC: 0.8025 | AUPR: 0.4976 | Acc: 95.04%
Internal test: AUC: 0.8135 | AUPR: 0.5449 | Acc: 94.84%
External test: AUC: 0.8101 | AUPR: 0.5081 | Acc: 94.82%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8795 | AUPR: 0.4465 | Acc: 83.56%
best model()
Valid:         AUC: 0.8795 | AUPR: 0.4465 | Acc: 83.56%
Internal test: AUC: 0.8894 | AUPR: 0.4857 | Acc: 84.38%
External test: AUC: 0.8787 | AUPR: 0.4288 | Acc: 82.52%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9181 | AUPR: 0.6532 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6521 | Acc: 95.25%
model(C=100.0)
Valid:         AUC: 0.9170 | AUPR: 0.6515 | Acc: 95.24%
best model(C=1.0)
Valid:         AUC: 0.9181 | AUPR: 0.6532 | Acc: 95.27%
Internal test: AUC: 0.9270 | AUPR: 0.6852 | Acc: 95.13%
External test: AUC: 0.9154 | AUPR: 0.6342 | Acc: 94.89%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8662 | AUPR: 0.5591 | Acc: 84.24%
model(criterion=log_loss)
Valid:         AUC: 0.8732 | AUPR: 0.5759 | Acc: 84.79%
model(criterion=entropy)
Valid:         AUC: 0.8732 | AUPR: 0.5759 | Acc: 84.79%
best model(criterion=log_loss)
Valid:         AUC: 0.8732 | AUPR: 0.5759 | Acc: 84.79%
Internal test: AUC: 0.8735 | AUPR: 0.5982 | Acc: 85.50%
External test: AUC: 0.8814 | AUPR: 0.5994 | Acc: 85.57%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9151 | AUPR: 0.6548 | Acc: 89.15%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6574 | Acc: 89.35%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9151 | AUPR: 0.6573 | Acc: 89.31%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9160 | AUPR: 0.6606 | Acc: 89.43%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9161 | AUPR: 0.6609 | Acc: 89.39%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9155 | AUPR: 0.6597 | Acc: 89.37%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9160 | AUPR: 0.6606 | Acc: 89.43%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9161 | AUPR: 0.6609 | Acc: 89.39%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9155 | AUPR: 0.6597 | Acc: 89.37%
best model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9161 | AUPR: 0.6609 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9151 | AUPR: 0.6438 | Acc: 95.22%
model(criterion=squared_error)
Valid:         AUC: 0.9153 | AUPR: 0.6448 | Acc: 95.28%
best model(criterion=squared_error)
Valid:         AUC: 0.9153 | AUPR: 0.6448 | Acc: 95.28%
Internal test: AUC: 0.9285 | AUPR: 0.6857 | Acc: 95.40%
External test: AUC: 0.9231 | AUPR: 0.6558 | Acc: 95.22%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9151 | AUPR: 0.6331 | Acc: 95.12%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9161 | AUPR: 0.6348 | Acc: 95.14%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9158 | AUPR: 0.6335 | Acc: 95.17%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9161 | AUPR: 0.6348 | Acc: 95.14%
Internal test: AUC: 0.9197 | AUPR: 0.6753 | Acc: 95.06%
External test: AUC: 0.9149 | AUPR: 0.6191 | Acc: 94.77%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)